In [76]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter, median_filter
from numpy.fft import fft2, fftshift
from tqdm import tqdm
from skimage.restoration import (
    denoise_tv_chambolle,
    denoise_bilateral,
    denoise_wavelet,
    estimate_sigma,
)

In [77]:
# Funzione per la trasformata di Fourier e la visualizzazione dello spettro
def fourier_transform(image):
    # Calcola la trasformata di Fourier
    f_transform = np.fft.fft2(image)
    f_shift = np.fft.fftshift(f_transform)
    
    # Calcola lo spettro di magnitudine
    magnitude_spectrum = 20 * np.log(np.abs(f_shift))
    
    return magnitude_spectrum




Traslo le immagini dal dominio spaziale al dominio delle frequenze attraverso la trasformata di Fourier

In [92]:
# Funzione per elaborare tutte le immagini nelle sotto-cartelle
def process_images_in_subfolders(input_folder, output_folder):
    # Trova tutte le immagini nei percorsi specificati
    image_files = []
    for root, dirs, files in os.walk(input_folder):
        for file_name in files:
            if file_name.endswith(".jpg"):
                image_files.append(os.path.join(root, file_name))

    # Inizializza la barra di avanzamento con il numero totale di immagini
    with tqdm(total=len(image_files), desc="Images processing...") as pbar:
        # Ciclo attraverso tutte le immagini e le elabora
        for image_path in image_files:
            # Carica l'immagine
            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            # Applico il filtro gaussiano per effettuare il denoising dell'immagine
            currentResidual = image - gaussian_filter(image, sigma=50)
            
            # Applica la trasformata di Fourier
            spectrum = fourier_transform(currentResidual)
            
            # Crea il percorso di output mantenendo la struttura delle sotto-cartelle
            relative_path = os.path.relpath(os.path.dirname(image_path), input_folder)
            output_subfolder = os.path.join(output_folder, relative_path)
            output_path = os.path.join(output_subfolder, os.path.basename(image_path))
            
            # Controlla se la sotto-cartella di output esiste, altrimenti lòa crea
            if not os.path.exists(output_subfolder):
                os.makedirs(output_subfolder)
            
            # Salva l'immagine nello spazio delle frequenze
            cv2.imwrite(output_path, spectrum)
            
            # Aggiorna la barra di avanzamento
            pbar.update(1)




Qui di seguito vengono salvati all'interno della directory di destinazione "biggan_spectrum" gli spettri delle immagini generate con il modello BigGAN

In [93]:
# Ottiene il percorso corrente dello script
current_dir = os.getcwd()

# Cartella contenente le immagini create con la BigGAN
input_folder = os.path.join(current_dir, "dataset", "big_gan", "big", "biggan-data")

# Cartella di destinazione per le immagini nello spazio delle frequenze; 
# ho creato la cartella biggan-spectrum, la quale mantiene la stessa gerarchia della cartella
# contenente le immagini prima della trasformazione
output_folder = os.path.join(current_dir, "dataset", "big_gan", "big", "biggan-spectrum")

# Controlla che la cartella di destinazione esiste, altrimenti la crea
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Elabora tutte le immagini nelle sotto-cartelle
process_images_in_subfolders(input_folder, output_folder)

Images processing...: 100%|██████████████████████████████████████████████████████| 10000/10000 [02:18<00:00, 72.06it/s]
